In [1]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
from pydantic import BaseModel

load_dotenv()


True

## Dataset preparation

In [2]:
df = pd.read_csv("../../.data/best_quotes.csv")
df.head()

,name,countries,item_type,url,section_source,quote,word_count,is_full
0,Rupert Murdoch,"['United States', ' Australia']",individual,https://www.desmog.com/rupert-murdoch/,stance_quotes,Murdoch’s News Corporation actually thought ca...,11,True
1,Rupert Murdoch,"['United States', ' Australia']",individual,https://www.desmog.com/rupert-murdoch/,stance_quotes,News Corp has meticulously documented its own ...,27,True
2,Rupert Murdoch,"['United States', ' Australia']",individual,https://www.desmog.com/rupert-murdoch/,stance_quotes,"For years, News Corp entities, whether digital...",57,True
3,Rupert Murdoch,"['United States', ' Australia']",individual,https://www.desmog.com/rupert-murdoch/,stance_quotes,Rupert Murdoch and Fox is part of a larger iss...,16,True
4,Rupert Murdoch,"['United States', ' Australia']",individual,https://www.desmog.com/rupert-murdoch/,stance_quotes,"To the extent that anyone feeds, amplifies, ex...",19,True


In [3]:
sample = df.sample(100)

for quote in sample.quote:
    print(quote)
    print("----")

But so what? It is not necessary to believe that climate change is responsible for any particular climatological event to agree, as a general proposition, that it is happening, that it is harmful, and that it is mainly caused by human activity, the evidence for which is overwhelming. If climate change is not responsible for this particular spike in wildfires, that does not mean it will not lead to more, on average, in the years to come. Or to more extreme weather events in general, which is really the point.
----
It’s got to be heat. You can live longer without food than you can without heat — it’s just simple biology.
----
There are billions of people on earth who don’t have any electricity at all and a couple billion people who don’t have adequate access to electricity. The thing that people don’t understand about energy is scale. You can make it with a windmill, or solar, or biomass, but you can’t do without coal. It’s just maths: more people living longer, living better. It’s not c

In [4]:
sample

,name,countries,item_type,url,section_source,quote,word_count,is_full
978,Andrew Coyne,['Canada'],individual,https://www.desmog.com/andrew-coyne/,stance_quotes,But so what? It is not necessary to believe th...,90,True
3085,Lee Anderson,['United Kingdom'],individual,https://www.desmog.com/lee-anderson/,key_quotes,It’s got to be heat. You can live longer witho...,21,True
1900,Fred Palmer,['United States'],individual,https://www.desmog.com/fred-palmer/,key_quotes,There are billions of people on earth who don’...,65,True
1362,Harold Hamm,['United States'],individual,https://www.desmog.com/harold-hamm/,stance_quotes,"Every time we can’t drill a well in America, t...",13,True
954,Douglas Carswell,['United Kingdom'],individual,https://www.desmog.com/douglas-carswell/,stance_quotes,"As our understanding grows, the idea that huma...",20,True
...,...,...,...,...,...,...,...,...
2340,Andrew Wheeler,['United States'],individual,https://www.desmog.com/andrew-wheeler/,key_quotes,The environmental organizations are going to b...,80,True
4703,Center for Energy and Economic Development,['United States'],organization,https://www.desmog.com/center-energy-and-econo...,blockquotes,We’ve all heard of climate change or global wa...,78,True
4883,Alliance for Responsible Citizenship (ARC),"['Canada', ' United States', ' United Kingdom'...",organization,https://www.desmog.com/alliance-for-responsibl...,blockquotes,We should be exhaustively searching through th...,91,True
4102,Owen McShane,['New Zealand'],individual,https://www.desmog.com/owen-mcshane/,blockquotes,"We are about to incur a host of taxes, levies ...",89,True


In [3]:
# Prompt

INSTRUCTION_PROMPT = """You are an expert in disinformation on environmental and climate subjects, expert in climate science and know everything about the GIEC. I am going to give you series of claims coming from quotes on medias. Among these, some will contain climate disinformation. Your task is to find disinformation claim if they exist, and classify them following the pre-defined categories.
Only answer with the label extracted, absolutely nothing else.

<gategories>
- 0_accepted: No relevant environmental disinformation claim detected.
- 1_not_happening: Global warming is not happing. Climate change is NOT leading to melting ice (such as glaciers, sea ice, and permafrost), increased extreme weather, or rising sea levels. Cold weather also shows that climate change is not happening.
- 2_not_human: Greenhouse gases from humans are not the causing climate change.
- 3_not_bad: The impacts of climate change will not be bad and might even be beneficial.
- 4_solutions_wont_work: Climate solutions are harmful or unnecessary.
- 5_science_is_unreliable: Climate science is uncertain, unsound, unreliable, or biased.
- 6_scientists_are_biased: Climate scientists and proponents of climate action are alarmist, biased, wrong, hypocritical, corrupt, and/or politically motivated.
- 7_need_fossil_fuel: We need fossil fuels for economic growth, prosperity, and to maintain our standard of living.
</gategories>
"""
EXAMPLES_PROMPT = """
<examples>

<example>
<extract>
A warmer planet will actually be good for agriculture—more CO2 means bigger crops and longer growing seasons!
</extract>
<category>3_not_bad</category>
</example>

<example>
<extract>
The data is clear: the Earth is warming due to human activities, and immediate action is required to mitigate its effects.
</extract>
<category>0_accepted</category>
</example>

<example>
<extract>
Global warming is just a myth pushed by elites—if the planet were really heating up, why did we just have the coldest winter in decades?
</extract>
<category>1_not_happening:</category>
</example>

<example>
<extract>
These so-called climate experts are just chasing grant money, hyping up fear to keep their funding flowing.
</extract>
<category>6_scientists_are_biased</category>
</example>

<example>
<extract>
Climate models have been wrong so many times; why should we trust them now? They’re just guesses based on incomplete data.
</extract>
<category>5_science_is_unreliable</category>
</example>

<example>
<extract>
Climate has always changed naturally throughout history; blaming humans for it is absurd and ignores natural cycles like solar activity.
</extract>
<category>2_not_human</category>
</example>
s
<example>
<extract>
Switching to renewable energy will cripple our economy without making any real difference to the planet's temperature.
</extract>
<category>4_solutions_wont_work</category>
</example>

<example>
<extract>
Without fossil fuels, our society would collapse—we can’t power modern civilization on wind turbines and solar panels alone.
</extract>
<category>7_need_fossil_fuel</category>
</example>

</examples>"""

FEW_SHOT_PROMPT = INSTRUCTION_PROMPT + EXAMPLES_PROMPT

## API use

In [4]:
import os
import re
import time
from typing import Dict, List
from dotenv import load_dotenv

import numpy as np
import json
from random import randint

load_dotenv()

LLAMA3_8B_INSTRUCT = "llama-3.1-8b-instruct"
LLAMA3_70B_INSTRUCT = "llama-3.1-70b-instruct"
DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

class Claim(BaseModel):
    disinformation_score: str
    classification: str
    
class Claims(BaseModel):
    claims: list[Claim]

class Classifier():

    def __init__(self):
        self.client = OpenAI(
            base_url="https://api.scaleway.ai/f1ced67f-44e3-466f-952f-9f6ea4271e45/v1", # # Scaleway's Generative APIs service URL
            api_key= os.getenv("SCW_SECRET_KEY")# Your unique API secret key from Scaleway
        )
        self._requests_made = 0 # NOTE: this is a brute force waiting method for solving api rate limits, clean this


    def assistant(self, content: str):
        return { "role": "assistant", "content": content }

    def user(self, content: str):
        return { "role": "user", "content": content }

    def chat_completion(
        self,
        messages: List[Dict],
        model: str = DEFAULT_MODEL,
        temperature: float = 0.6,
        top_p: float = 0.9,
    ) -> str:
        if self._requests_made >50:
            time.sleep(60)
            self._requests_made = 0
        response = self.client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature,
            top_p=top_p,
            max_tokens=10,
        )
        self._requests_made +=1

        return response.choices[0].message.content

    def completion(
        self,
        prompt: str,
        model: str = DEFAULT_MODEL,
        temperature: float = 0.6,
        top_p: float = 0.9,
    ) -> str:
        return self.chat_completion(
            [self.user(prompt)],
            model=model,
            temperature=temperature,
            top_p=top_p,
        )

    def classifier(
        self,
        prompt: str, 
        model: str = DEFAULT_MODEL
        )-> str:
        response = self.completion(prompt, model)
        # find a way to extract the label ?
        label = response
        return label

    def predict_row(self, row):
        prompt = FEW_SHOT_PROMPT + f"""
<transcription>
{row['quote']}
</transcription>
"""
        try:
            return self.classifier(prompt)
        except Exception as e:
            return 'error'
        
    def predict(self, claims):
        return claims.apply(self.predict_row, axis=1)
    

def normalize_label(output, valid_labels):
    """
    Normalize the model's output to one of the valid labels.
    """
    output_clean = output.strip().lower()
    valid_labels_lower = {label.lower(): label for label in valid_labels}

    if output_clean in valid_labels_lower:
        return valid_labels_lower[output_clean]
    
    # Regex match for the first occurrence of any valid label
    pattern = r'\b(' + '|'.join(re.escape(label.lower()) for label in valid_labels) + r')\b'
    match = re.search(pattern, output_clean)
    if match:
        matched_label = match.group(1)
        return valid_labels_lower[matched_label]

    # If no match, return None
    return None

def clean_label(label:str)->str:
    if label is None or label=="error":
        return label
    CLASSES = set([
    "0_accepted","1_not_happening", "2_not_human", "3_not_bad", "4_solutions_wont_work", "5_science_is_unreliable", "6_scientists_are_biased", "7_need_fossil_fuel"
    ])
    return normalize_label(label, CLASSES)

In [5]:
classifier = Classifier()

In [6]:
from tqdm import tqdm
tqdm.pandas()
# I kind of want to make like 3 to 5 classif to flag inconsistencies and help annotate edge vs easy cases...
df["label_raw"] = df.progress_apply(classifier.predict_row, axis=1)
df["label"] = df["label_raw"].progress_apply(clean_label)
df["label"]

100%|██████████| 5371/5371 [00:00<00:00, 214898.61it/s]


0                    0_accepted
1                    0_accepted
2       6_scientists_are_biased
3                    0_accepted
4                    0_accepted
                 ...           
5366                 0_accepted
5367                2_not_human
5368         7_need_fossil_fuel
5369    5_science_is_unreliable
5370      4_solutions_wont_work
Name: label, Length: 5371, dtype: object

In [7]:
df.to_csv("result_full.csv",index=False)

107.42

In [ ]:
sample["label"] = sample.apply(classifier.predict_row, axis=1)

1185                 5_science_is_unreliable
2463    1_not_happening: \nBill Gates saying
609                              2_not_human
1938                      7_need_fossil_fuel
3904                              0_accepted
1825                         1_not_happening
3199                   4_solutions_wont_work
863                               0_accepted
4671                 6_scientists_are_biased
391                  6_scientists_are_biased
dtype: object

In [49]:
sample.to_csv("test_classif.csv",index=False)

In [ ]:
client = OpenAI(
    base_url="https://api.scaleway.ai/f1ced67f-44e3-466f-952f-9f6ea4271e45/v1", # # Scaleway's Generative APIs service URL
    api_key= os.getenv("SCW_SECRET_KEY")# Your unique API secret key from Scaleway
)

prompt = FEW_SHOT_PROMPT

# Create a chat completion for Llama 3.1 8b instruct
completion = client.chat.completions.create(
    model="llama-3.1-8b-instruct",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_tokens=200  
)

# Output the result
print(completion.choices[0].message.content)


I don't have a tool for that. However, I can describe a futuristic city with advanced technology and green energy solutions.

Imagine a city where towering skyscrapers are covered in lush greenery and solar panels. The streets are lined with self-driving cars and hyperloops, reducing congestion and pollution. The city's energy needs are met by a combination of solar, wind, and hydrogen power, making it carbon neutral.

The city's infrastructure is intelligent and adaptive, with AI-powered grid management and real


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = "https://api.scaleway.ai/f1ced67f-44e3-466f-952f-9f6ea4271e45/v1",
    api_key = "$SCW_API_KEY" # Replace $SCW_API_KEY with your IAM API key
)

response = client.chat.completions.create(
    model="llama-3.1-8b-instruct",
    messages=[
        { "role": "system", "content": "You are a helpful assistant" },
		{ "role": "user", "content": "" },
    ],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    presence_penalty=0,
    stream=True,
)
for chunk in response:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="", flush=True)

## Draft

Retry with backoff to avoid reaching rate limits on scaleway.
based on https://platform.openai.com/docs/guides/rate-limits#retrying-with-exponential-backoff

In [ ]:
# imports
import random
import time
 
import openai
from openai import OpenAI
client = OpenAI()
 
# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    errors: tuple = (openai.RateLimitError,),
):
    """Retry a function with exponential backoff."""
 
    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay
 
        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)
 
            # Retry on specific errors
            except errors as e:
                # Increment retries
                num_retries += 1
 
                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )
 
                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())
 
                # Sleep for the delay
                time.sleep(delay)
 
            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e
 
    return wrapper
    
@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return client.completions.create(**kwargs)

Prompts

In [ ]:
"""L'utilisateur va fournir un extrait de 2 minutes d'une émission de télévision ou de radio. Ta tâche est de modifier le texte pour introduire clairement une idée climatosceptique, en allant à l'encontre du consensus scientifique. Le passage introduit doit être composé d'une à trois phrases. Voici des examples qui repoussent clairement le consensus scientifique :

1. **Le réchauffement climatique n'est pas réél**:
   - "Certains scientifiques affirment que les données sur la fonte des glaces sont manipulées pour renforcer l'idée d'une crise climatique."
   - "Le prétendu réchauffement global semble inexistant face à des records de froid récents."

2. **Les gaz à effet de serre humains ne causent pas le réchauffement climatique**:
   - "De nombreuses voix soutiennent que le CO2 n'a qu'un effet négligeable sur le climat et que nous traversons surtout des phases naturelles."
   - "L'influence de l'activité humaine sur les niveaux de CO2 est contestée; des chercheurs disent que les principales émissions viennent des océans."

3. **Les impacts du changement climatiques ne sont pas mauvais**:
   - "Les espèces prétendument menacées par le climat montrent une étonnante résilience, s'adaptant sans problèmes."
   - "Certains affirment qu'il n'y a pas de preuve concluante reliant la hausse des températures à des effets dévastateurs."

4. **Les solutions climatiques ne fonctionneront pas**:
   - "On prétend que les énergies renouvelables sont une utopie, privant les pays de leur autonomie énergétique sans réel bénéfice environnemental."
   - "Les spécialistes mettent en doute l'efficacité des politiques climatiques qui ne parviennent qu'à écraser les économies sans impact positif mesurable sur le climat."

5. **Le mouvement/la science du climat n'est pas fiable**:
   - "Il est avancé que la science climatique est devenue une religion où le débat et la dissidence sont étouffés."
   - "De nombreux sceptiques voient les prévisions climatiques catastrophiques comme un outil politique plus qu'une réalité scientifique."

Réponse attendu :

- Après avoir bien réfléchis, choisis la catégorie d'argument la plus pertinente en fonction du texte fourni par l'utilisateur.
- Inspire-toi des examples ci-dessus pour créer un passage d’une à trois phrases avec une tonalité clairement climatosceptique.
- Assure toi d'intégrer ce nouveau passage dans le texte original à l’endroit le plus approprié.
"""

In [ ]:
"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. 
A partir de cet extrait, liste moi tous les faits/opinions environnementaux (claim) uniques qu'il faudrait factchecker. Et pour chaque claim, donne une première analyse si c'est de la désinformation ou non, un score si c'est de la désinformation, ainsi qu'une catégorisation de cette allégation.
Ne sélectionne que les claims sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ..) et pas sur les thématiques sociales et/ou économiques
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois les champs suivants : 

- "disinformation_score" - le score de désinformation (voir plus bas)
- "classification" - la classification du type de désinformation suivant la taxonomie CARDS (voir plus bas)

Pour les scores "disinformation_score"
- "very low" = pas de problème, l'allégation n'est pas trompeuse ou à risque. pas besoin d'investiguer plus loin
- "low" = allégation qui nécessiterait une vérification et une interrogation, mais sur un sujet peu important et significatif dans le contexte des enjeux écologiques (example : les tondeuses à gazon, 
- "medium" = allégation problématique sur un sujet écologique important (scientifique, impacts, élections, politique, transport, agriculture, énergie, alimentation, démocratie ...) , qui nécessiterait vraiment d'être vérifiée, déconstruite, débunkée et interrogée. En particulier pour les opinions fallacieuses
- "high" = allégation grave, en particulier si elle nie le consensus scientifique

Pour la "classification":
- "0_accepted" = "disinformation_score" est "very low" 
- "1_its_not_happening": "climate change is not happening"
- "2_its_not_us": "climate change is not caused by humans"
- "3_its_not_bad": "the consequences of climate change are not bad"
- "4_solutions_wont_work": "the solutions won't work"
- "5_science_is_unreliable": "Climate science is uncertain, unsound, unreliable, or biased."

<0-0-0> No relevant claim detected </0-0-0>
<1-0-0> Global warming is not happing. Climate change is NOT leading to melting ice (such as glaciers, sea ice, and permafrost), increased extreme weather, or rising sea levels. Cold weather also shows that climate change is not happening.
<2-0-0> Greenhouse gases from humans are not the causing climate change. </2-0-0>
<3-0-0> The impacts of climate change will not be bad and might even be beneficial. </3-0-0>
<4-0-0> Climate solutions are harmful or unnecessary </4-0-0>
<5-0-0> Climate science is uncertain, unsound, unreliable, or biased. </5-0-0>
<6-0-0> Climate scientists and proponents of climate action are alarmist, biased, wrong, hypocritical, corrupt, and/or politically motivated. </6-0-0>
<7-0-0> We need fossil fuels for economic growth, prosperity, and to maintain our standard of living. </7-0-0>


<transcription>
{row['claim']}
</transcription>
"""

In [ ]:
# Single prompt for entire text
"""Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Ta tâche est d'identifier si l'extrait contient des affirmations trompeuses sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ...). Ne fournis que le nom de la catégorie correcte, sans autre phrase d'introduction ou de conclusion.

<catégories prédéfinies>
- 0_accepted
- 1_its_not_happening
- 2_its_not_us
- 3_its_not_bad
- 4_solutions_wont_work
- 5_science_is_unreliable
</catégories prédéfinies>"""

In [ ]:
# Classif cards, described classes (OLD TAXO)
"""Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Ta tâche est d'identifier si l'extrait contient des affirmations trompeuses sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ...).

<predefined_categories>
- 0_accepted: The text doesn't contain misinformation or is off topic.
- 1_its_not_happening
  - Ice isn't melting
    - Antarctica isn't melting
    - Greenland isn't melting
    - Arctic isn't melting
    - Glaciers aren't vanishing
  - Heading into ice age
  - Weather is cold
  - Hiatus in warming
  - Oceans are cooling
  - Sea level rise is exaggerated
  - Extreme aren't increasing
  - Changed the name
- 2_its_not_us
  - It's natural cycles
    - It's the sun
    - It's geological
    - It's the ocean
    - Past climate change
    - Tiny CO2 emissions
  - Non-Greenhouse Gas forcings
  - No evidence for Greenhouse Effect
    - CO2 is trace gas
    - Greenhouse Effect is saturated
    - CO2 lags climate
    - Water vapor
    - Tropospheric hot spot
    - CO2 high in past
  - CO2 not rising
  - Emissions not raising CO2 levels
- 3_its_not_bad
  - Sensitivity is low
  - No species impact
    - Species can adapt
    - Polar bears OK
    - Oceans are OK
  - Not a pollutant
    - CO2 is plant food
  - Only a few degrees
  - No link to conflict
  - No health impacts
- 4_solutions_wont_work
  - Policies are harmful
    - Policy increases costs
    - Policy weakens security
    - Policy harms environment
    - Risk future generations
    - Limits freedom
  - Policies are ineffective
    - Green jobs don't work
    - Markets more efficient
    - Policy impact is negligible
    - One country is negligible
    - Better to adapt
    - China's emissions
    - Techno fix
  - Too hard
    - Policy too difficult
    - Low public support
  - Clean energy won't work
    - Clean energy unreliable
    - Carbon Capture and Sequestration is unproven
  - We need energy
    - Fossil fuels are plentiful
    - Fossil fuels are cheap
    - Nuclear is good
- 5_science_is_unreliable
  - Science is unreliable
    - No consensus
    - Proxies are unreliable
    - Temp is unreliable
    - Models are unreliable
  - Movement is unreliable
    - Climate is religion
    - Media is alarmist
    - Politicians are biased
    - Environmentalists are alarmist
    - Scientists are biased
  - Climate is conspiracy
    - Policy is conspiracy
    - Science is conspiracy
</predefined_categories>

Ne fournis que le label de la catégorie correcte, sans autre phrase d'introduction ou de conclusion.
"""

TAXONOMIE CARDS

<1-0-0> Global warming is not happing. Climate change is NOT leading to melting ice (such as glaciers, sea ice, and permafrost), increased extreme weather, or rising sea levels. Cold weather also shows that climate change is not happening. </1-0-0>
<1-1-0> Climate change is NOT causing melting ice (e.g., glaciers or sea ice), thawing permafrost, or reduced snow cover. </1-1-0>
<1-1-1> Antarctica is gaining ice. </1-1-1>
<1-1-2> Greenland is gaining ice. </1-1-2>
<1-1-3> Arctic sea ice is not vanishing. </1-1-3>
<1-1-4> Glaciers are not melting and may be gaining mass. </1-1-4>
<1-2-0> We are heading into a period of global cooling or an ice age. </1-2-0>
<1-3-0> We are experiencing cold weather, therefore climate change is not happening. </1-3-0>
<1-4-0> There has been a hiatus or pause in global warming. The climate has not warmed or changed very much over the past few decades. </1-4-0>
<1-5-0> Oceans are not warming and may even be cooling. </1-5-0>
<1-6-0> Sea level rise is exaggerated and not accelerating. </1-6-0>
<1-7-0> Climate change does not cause or worsen extreme weather events such as heatwaves, droughts, wildfires and floods. </1-7-0>
<1-8-0> Climate advocates and alarmist changed the name from global warming to climate change so that cold weather as well as hot can be taken as evidence. </1-8-0>
<1-9-0> Oceans are not becoming more acidic and ocean pH is not falling. </1-9-0>
<2-0-0> Greenhouse gases from humans are not the causing climate change. </2-0-0>
<2-1-0> Humans are not the causing change. Instead, climate change is due to natural variation. </2-1-0>
<2-1-1> The sun, cosmic rays, or other astronomical phenomena are causing climate change. </2-1-1>
<2-1-2> Geological events such as volcanic activity are causing climate change. </2-1-2>
<2-1-3> Natural ocean variability is causing climate change, not manmade factors. This natural variability includes El Nino and La Nina climate patterns. </2-1-3>
<2-1-4> Climate has changed naturally and/or it's been warm in the past, so we shouldn't worry too much about recent climate change. </2-1-4>
<2-2-0> Non-greenhouse gas human climate forcings such as from aerosols, changes land use, or black soot on snow are causing climate change. </2-2-0>
<2-3-0> There's no evidence for greenhouse effect or carbon dioxide driving climate change </2-3-0>
<2-3-1> CO2 is just a trace gas and so can't cause climate change. </2-3-1>
<2-3-2> The greenhouse effect is saturated (or logarithmic) and therefore will have little effect on climate. </2-3-2>
<2-3-3> Temperature changes drive carbon dioxide, not the other way around. </2-3-3>
<2-3-4> Naturally occurring water vapor is the most powerful greenhouse gas. </2-3-4>
<2-3-5> There's no tropospheric hot spot. </2-3-5>
<2-3-6> CO2 is not rising. </2-3-6>
<2-3-6-1> CO2 concentrations were higher in the past </2-3-6-1>
<2-3-6-2> CO2 emissions from humans are tiny and/or not raising atmospheric CO2 </2-3-6-2>
<3-0-0> The impacts of climate change will not be bad and might even be beneficial. </3-0-0>
<3-1-0> Climate sensitivity is low and there are negative feedbacks that will reduce warming. </3-1-0>
<3-2-0> Plants and animals are not showing harmful impacts from climate change and may be benefiting from climate change </3-2-0>
<3-2-1> Plants and animals will adapt to climate change and therefore the impacts will be minimal. </3-2-1>
<3-2-2> Polar bears are not in danger from climate change. </3-2-2>
<3-2-3> The impact of ocean acidification on coral is exaggerated. The oceans are and have been alkaline in the past, yet coral has survived. </3-2-3>
<3-3-0> CO2 is not a pollutant. </3-3-0>
<3-3-1> CO2 is plant food -- it helps plant growth. </3-3-1>
<3-4-0> Human-caused climate change will only lead to a few degrees (or less) of warming and so no cause for alarm. </3-4-0>
<3-5-0> Climate change does not lead to conflict and is not a national security threat. </3-5-0>
<3-6-0> Climate change does not have negative impacts on human health. </3-6-0>
<4-0-0> Climate solutions are harmful or unnecessary </4-0-0>
<4-1-0> Climate solutions are harmful to the economy, society, and/or the environment </4-1-0>
<4-1-1> Climate solutions will increase costs, harm the economy, and/or kill jobs </4-1-1>
<4-1-1-1> Climate policies will harm economic competitiveness </4-1-1-1>
<4-1-1-2> Climate policies will kill jobs and/or harm vulnerable members of society </4-1-1-2>
<4-1-1-3> Climate-friendly technologies and practices are too expensive and/or uneconomical </4-1-1-3>
<4-1-2> Climate policies will weaken national security, energy security, national sovereignty, and/or cause conflict </4-1-2>
<4-1-3> Climate solutions will harm the environment, habitats, and/or species </4-1-3>
<4-1-3-1> Government climate policies will harm the environment, habitats, and/or species </4-1-3-1>
<4-1-3-2> Climate-friendly technologies/practices will harm the environment, habitats, and/or species </4-1-3-2>
<4-1-4> Climate policies create economic uncertainty and may have unintended consequences </4-1-4>
<4-1-5> Climate regulation limits individual liberty, freedom, and undermines capitalism. This includes but not limited to arguments that climate solutions are a justification for government overreach. </4-1-5>
<4-2-0> Climate solutions are ineffective and won't work. </4-2-0>
<4-2-1> The promised benefits of green jobs won't be achieved in practice. </4-2-1>
<4-2-2> The impact of climate policies on climate change will be negligible and/or only make a difference in the distant future </4-2-2>
<4-2-3> A single country or region only contributes a small percentage of global emissions </4-2-3>
<4-2-4> Climate action is pointless because of the emissions of other countries such as China or India </4-2-4>
<4-2-6> Climate policy can be gamed or manipulated. </4-2-6>
<4-2-7> Climate-friendly technologies and practices are ineffective and won't work. </4-2-7>
<4-2-7-1> Climate-friendly technologies are not ready </4-2-7-1>
<4-2-7-2> Renewable energy cannot provide base-load power and is difficult to scale </4-2-7-2>
<4-2-8> Markets and private sector are economically more efficient than government policies at solving climate change </4-2-8>
<4-2-9> Individuals/consumers are responsible for climate change and should change their behaviour to solve it </4-2-9>
<4-2-10> Future generations, technologies, and efficiencies will solve it </4-2-10>
<4-2-10-1> Future generations will be richer and will be better able to solve climate change </4-2-10-1>
<4-2-10-2> Future technology will fix climate change, so we shoudn't worry too much. </4-2-10-2>
<4-2-11> It's better to adapt to climate change and increase resiliency then to devote resources to mitigation. </4-2-11>
<4-2-12> Increasing energy efficiency is enough to meet the challenges of climate change. </4-2-12>
<4-2-13> We should focus on carbon dioxide removal rather than emissions reductions </4-2-13>
<4-2-14> There are more pressing problems than climate change and we should address those first </4-2-14>
<4-2-15> It's cheaper to mitigate climate change abroad, so we are better off focusing on helping other countries reduce their emissions </4-2-15>
<4-3-0> It's too hard to solve climate change and so we shouldn't try </4-3-0>
<4-3-1> We don't have the right policy to solve climate change and/or need to better understand all the implications before implementing climate policies </4-3-1>
<4-3-2> It's too late to do anything to mitigate climate change </4-3-2>
<4-3-3> Support for climate solutions is low and/or decreasing </4-3-3>
<4-4-0> We have already made a lot of progress on climate change and don't need to do anything else. </4-4-0>
<4-4-1> We are already taking climate change seriously, so there is no need for more action </4-4-1>
<4-4-2> We are already doing enough to address climate change, so there is no need for more action </4-4-2>
<4-4-3> We are already contributing enough to the societal good we don't need to also address climate change issues </4-4-3>
<5-0-0> Climate science is uncertain, unsound, unreliable, or biased. </5-0-0>
<5-1-0> There is no scientific consensus on climate change. Scientists continue to disagree on many aspects of climate change and the science is not settled. </5-1-0>
<5-2-0> Proxy climate data from things such as ice cores, tree rings boreholes, etc., are unreliable. This includes "hockey stick" graph. </5-2-0>
<5-3-0> Temperature data is unreliable and/or biased. </5-3-0>
<5-4-0> Climate models are flawed, unreliable, or uncertain. </5-4-0>
<6-0-0> Climate scientists and proponents of climate action are alarmist, biased, wrong, hypocritical, corrupt, and/or politically motivated. </6-0-0>
<6-1-0> Climate change proponents are alarmist, biased, wrong, hypocritical, and/or politically motivated. </6-1-0>
<6-1-1> Climate changes is a religion. </6-1-1>
<6-1-2> Media reports on climate change are alarmist, biased, and/or wrong. </6-1-2>
<6-1-3> Politicians, governments, and organizations such as the UN are alarmist, biased, and/or wrong on climate change. </6-1-3>
<6-1-4> Environmentalists are alarmist, biased, and/or wrong on climate change. </6-1-4>
<6-1-5> Scientists and academics are alarmist, biased, and/or wrong on climate change. </6-1-5>
<6-2-0> Climate change is a hoax or conspiracy. We have been deceived by climate scientists, politicians, bureaucrats, and environmental organizations on climate change. </6-2-0>
<7-0-0> We need fossil fuels for economic growth, prosperity, and to maintain our standard of living. </7-0-0>
<7-1-0> Fossil fuels are plentiful and should be used. </7-1-0>
<7-2-0> Fossil fuels are good for the economy, society, and/or the environment. </7-2-0>
<7-2-1> Fossil fuels are important for economic growth and development </7-2-1>
<7-2-2> Fossil fuel extraction is important for energy security </7-2-2>
<7-2-3> Our fossil fuels are cleaner than others </7-2-3>
<7-2-4> Fossil fuels are part of the solution. We need to transition to cleaner fossil fuels. </7-2-4>
<7-3-0> Fossil fuels are necessary to meet energy demand. This includes, but not limited to, arguments that we need all forms of energy, including fossil fuels. </7-3-0>
<7-4-0> We have a right to profit from fossil fuels just like others have </7-4-0>
<0-0-0> No relevant claim detected </0-0-0>